In [14]:
import os
import glob

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import ray

# 바이살라 루프트 자료 병합

In [71]:
LuftFileList = glob.glob("../DATA/MWIS_2023_NDJF_OBS/*MARWIS*csv")
VaisalaFileList = glob.glob("../DATA/MWIS_2023_NDJF_OBS/*move4*csv")

luft_total_list = []
for luft in LuftFileList:
    luft_df = pd.read_csv(luft, low_memory=False)
    luft_total_list.append(luft_df)
luft_total_df = pd.concat(luft_total_list)

vi_total_list = []
for vi in VaisalaFileList:
    vi_df = pd.read_csv(vi, low_memory=False)
    vi_total_list.append(vi_df)
vi_total_df = pd.concat(vi_total_list)

def vaisala_road(s_status):
    vaisala_road_state = {
        '1': "Dry",
        '2': "Moist",
        '3': "Wet",
        '5': "Frost",
        '6': "Snow",
        '7': "Ice",
        '9': "Slush",
    }

    if s_status not in vaisala_road_state:
        return "error"
    return vaisala_road_state[s_status]

vi_total_df["s_status_txt"] = vi_total_df["s_status"].apply(vaisala_road)

drop_column_list = ['gdirection1','gdirection2','digitalcomp_x','digitalcomp_y','digitalcomp_z',
                    'ta2','rh2','loggerta', 'batteryvolt', 'rev1', 'rev2']
vi_total_df.drop(drop_column_list, axis = 1, inplace=True)

# Extract components of the date and time
year = vi_total_df['gdate'].astype(str).str[-6:-4].apply(lambda x: '20' + x if len(x) == 2 else x)  # Handling two-digit year
month = vi_total_df['gdate'].astype(str).str[-4:-2]
day = vi_total_df['gdate'].astype(str).str[-2:]

hour = vi_total_df['gtime'].astype(str).str[-6:-4].str.zfill(2)
minute = vi_total_df['gtime'].astype(str).str[-4:-2]
second = vi_total_df['gtime'].astype(str).str[-2:]

# Combine extracted components into a datetime string and convert to datetime
vi_total_df['TIMESTAMP'] = pd.to_datetime(
    year + '-' + month + '-' + day + ' ' + hour + ':' + minute + ':' + second,
    format='%Y-%m-%d %H:%M:%S'
)

vi_total_df['TIMESTAMP'] = pd.to_datetime(vi_total_df['TIMESTAMP'])
luft_total_df['TIMESTAMP'] = pd.to_datetime(luft_total_df['TIMESTAMP'])

total_df = pd.merge(vi_total_df, luft_total_df, on='TIMESTAMP', how='outer', suffixes=('_vi', '_luft'))

total_df.replace({
    '/': '',
    '//////': np.nan,
    '////': np.nan,
    '///////': np.nan
}, inplace=True)


# total_df.to_csv("../DATA/CONCAT/luft_vi_2023_2024.csv", index=None)



In [75]:
total_1 = total_df

In [80]:
# # NaN, 무한대 값을 이전 데이터로 채우기
# total_df[['glatitude','glongitude']]=total_df[['glatitude','glongitude']].astype(float)
# total_df[['glatitude','glongitude']]=total_df[['glatitude','glongitude']].replace([np.inf, -np.inf], np.nan, inplace=True)
# total_df[['glatitude','glongitude']]=total_df[['glatitude','glongitude']].replace("NaN", np.nan, inplace=True)
# total_df[['glatitude','glongitude']]=total_df[['glatitude','glongitude']].fillna(method='ffill', inplace=True)

# NaN, 무한대 값을 이전 데이터로 채우기
# total_1['glatitude']=total_1['glatitude'].astype(float)
#total_df['glatitude']=total_1['glatitude'].replace([np.inf, -np.inf], np.nan, inplace=True)
# total_1['glatitude']=total_1['glatitude'].replace("NaN", np.nan, inplace=True)

# # NaN, 무한대 값을 이전 데이터로 채우기
# total_df['glongitude']=total_df['glongitude'].astype(float)
# total_df['glongitude']=total_df['glongitude'].replace([np.inf, -np.inf], np.nan, inplace=True)
# total_df['glongitude']=total_df['glongitude'].replace("NaN", np.nan, inplace=True)
# total_df['glongitude']=total_df['glongitude'].fillna(method='ffill', inplace=True)

In [79]:
total_1

,vehicle,gdate,gtime,glatitude,glongitude,galtitude,vsp,rws,rwd,tws,...,R_Tem,R_Dew,R_Hum,R_WH,R_Sta,R_Ice,R_Fric,R_Sta_txt,BattV,PTemp
0,A,231117.0,65401.0,37.516632,127.439939,55.2,0.0,0.5,336.00,0.49,...,-0.323289,-1.210844,92.73611,0.041476,5.0,0.000000e+00,0.808356,Chemically wet,11.99,17.76652
1,A,231117.0,65402.0,37.516632,127.439939,55.2,0.0,0.5,337.00,0.46,...,-0.364372,-1.210844,93.70230,0.039128,5.0,0.000000e+00,0.808801,Chemically wet,12.00,17.76652
2,A,231117.0,65403.0,37.516632,127.439939,55.2,0.0,0.4,334.00,0.39,...,-0.282198,-1.219853,93.95007,0.041893,5.0,0.000000e+00,0.808278,Chemically wet,12.00,17.76652
3,NaN,NaN,NaN,37.516632,127.439939,NaN,NaN,NaN,NaN,NaN,...,-0.405463,-1.219853,93.49349,0.041732,5.0,0.000000e+00,0.808308,Chemically wet,12.00,17.76652
4,A,231117.0,65405.0,37.516633,127.439938,55.4,0.0,0.3,328.00,0.33,...,-0.237178,-1.210038,94.23958,0.040386,5.0,0.000000e+00,0.808561,Chemically wet,11.99,17.79138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943543,A,240223.0,124001.0,37.460904,126.890301,21.0,11.2,11.4,357.00,4.17,...,15.365010,-2.062749,30.24125,0.026036,1.0,3.082857e-44,0.811540,Damp,11.93,26.96393
943544,A,240223.0,124002.0,37.461070,126.890166,20.2,10.9,11.5,357.00,4.17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
943545,A,240223.0,124003.0,37.461148,126.890090,20.6,10.4,11.5,357.00,4.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
943546,A,240223.0,124004.0,37.461231,126.890008,23.1,11.1,11.4,356.00,4.13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
total_df['glatitude']=total_df['glatitude'].ffill() 
total_df['glongitude']=total_df['glongitude'].ffill() 

# AWS 정보 데이터프레임 읽기 및 k-d 트리 생성
aws_info_df = pd.read_csv("../DATA/AWS/META_관측지점정보.csv", encoding='cp949')
aws_coord = aws_info_df.drop_duplicates(subset=["지점"])[['지점', '위도', '경도']]
aws_coord.columns = ['site', 'latitude', 'longitude']

# 위도와 경도 점들을 배열로 생성
points = np.array(list(zip(aws_coord['latitude'], aws_coord['longitude'])))
tree = KDTree(points)

# 질의 점들 생성
query_points = total_df[['glatitude', 'glongitude']].to_numpy()

# 가장 가까운 점들 찾기
distances, indices = tree.query(query_points)

# 가장 가까운 점의 사이트 정보 가져오기
nearest_sites = aws_coord.iloc[indices]['site'].values

# total_df에 AWS 사이트 정보 추가
total_df['aws_site'] = nearest_sites

In [82]:
total_df

,vehicle,gdate,gtime,glatitude,glongitude,galtitude,vsp,rws,rwd,tws,...,R_Dew,R_Hum,R_WH,R_Sta,R_Ice,R_Fric,R_Sta_txt,BattV,PTemp,aws_site
0,A,231117.0,65401.0,37.516632,127.439939,55.2,0.0,0.5,336.00,0.49,...,-1.210844,92.73611,0.041476,5.0,0.000000e+00,0.808356,Chemically wet,11.99,17.76652,326
1,A,231117.0,65402.0,37.516632,127.439939,55.2,0.0,0.5,337.00,0.46,...,-1.210844,93.70230,0.039128,5.0,0.000000e+00,0.808801,Chemically wet,12.00,17.76652,326
2,A,231117.0,65403.0,37.516632,127.439939,55.2,0.0,0.4,334.00,0.39,...,-1.219853,93.95007,0.041893,5.0,0.000000e+00,0.808278,Chemically wet,12.00,17.76652,326
3,NaN,NaN,NaN,37.516632,127.439939,NaN,NaN,NaN,NaN,NaN,...,-1.219853,93.49349,0.041732,5.0,0.000000e+00,0.808308,Chemically wet,12.00,17.76652,326
4,A,231117.0,65405.0,37.516633,127.439938,55.4,0.0,0.3,328.00,0.33,...,-1.210038,94.23958,0.040386,5.0,0.000000e+00,0.808561,Chemically wet,11.99,17.79138,326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943543,A,240223.0,124001.0,37.460904,126.890301,21.0,11.2,11.4,357.00,4.17,...,-2.062749,30.24125,0.026036,1.0,3.082857e-44,0.811540,Damp,11.93,26.96393,417
943544,A,240223.0,124002.0,37.461070,126.890166,20.2,10.9,11.5,357.00,4.17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,417
943545,A,240223.0,124003.0,37.461148,126.890090,20.6,10.4,11.5,357.00,4.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,417
943546,A,240223.0,124004.0,37.461231,126.890008,23.1,11.1,11.4,356.00,4.13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,417


# 센서 자료 AWS자료 병합

In [22]:
total_df = total_df[:100]
from scipy.spatial import KDTree
import numpy as np
# NaN, 무한대 값을 이전 데이터로 채우기
total_df[['glatitude','glongitude']].replace([np.inf, -np.inf], np.nan, inplace=True)
total_df[['glatitude','glongitude']].fillna(method='ffill', inplace=True)

def merge_coord(glatitude, glongitude):
    # AWS 정보 데이터프레임 읽기
    aws_info_df = pd.read_csv("../DATA/AWS/META_관측지점정보.csv", encoding='cp949')
    # 중복된 지점 제거 후 위도, 경도 추출
    aws_coord = aws_info_df.drop_duplicates(subset=["지점"])[['지점', '위도', '경도']]
    aws_coord.columns = ['site', 'latitude', 'longitude']
    
    # 위도와 경도 점들을 배열로 생성
    points = np.array(list(zip(aws_coord['latitude'], aws_coord['longitude'])))
    
    # k-d 트리 생성
    tree = KDTree(points)
    
    # 질의 점 생성
    query_point = np.array([glatitude, glongitude])
    
    # 가장 가까운 점 찾기
    distance, index = tree.query(query_point)
    
    # 가장 가까운 점의 사이트 정보 가져오기
    site = aws_coord.iloc[index]['site']
    
    return site

total_df["aws_site"] = total_df.apply(lambda row: merge_coord(row['glatitude'], row['glongitude']), axis=1) 

C:\Users\user\AppData\Local\Temp\ipykernel_23024\3320728029.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_df[['glatitude','glongitude']].replace([np.inf, -np.inf], np.nan, inplace=True)
C:\Users\user\AppData\Local\Temp\ipykernel_23024\3320728029.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  total_df[['glatitude','glongitude']].fillna(method='ffill', inplace=True)
C:\Users\user\AppData\Local\Temp\ipykernel_23024\3320728029.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_df[['glatitude','glongitude']].f

In [23]:
# luft_total_df.to_csv("../DATA/CONCAT/luft_2023_2024.csv", index=None)
# vi_total_df.to_csv("../DATA/CONCAT/visala_2023_2024.csv", index=None)